<a href="https://colab.research.google.com/github/gdsaikrishna/My-Projects/blob/master/Web_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
#Scraping News data from POLITIFACT

In [37]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [38]:
#Lists to store scraped data
authors = []
dates = []
statements = []
sources = []
targets = []

In [39]:
#Function to scrape
def scrape_website(page_number):
    page_num = str(page_number)
    URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.text, 'html.parser')
    #author location
    statement_footer = soup.find_all('footer', attrs={'class':'m-statement__footer'})
    #statement location
    statement_quote = soup.find_all('div', attrs={'class':'m-statement__quote'})
    #source location
    statement_meta = soup.find_all('div', attrs={'class':'m-statement__meta'})
    #image(target) location
    target = soup.find_all('div', attrs = {'class':'m-statement__meter'})

    for i in statement_footer:
        link1 = i.text.strip()
        name_and_date = link1.split()
        first_name = name_and_date[1]
        last_name = name_and_date[2]
        full_name = first_name+' '+last_name
        month = name_and_date[4]
        day = name_and_date[5]
        year = name_and_date[6]
        date= month+' '+day+' '+year
        dates.append(date)
        authors.append(full_name)

    for i in statement_quote:
        link2 = i.find_all('a')
        statement_text = link2[0].text.strip()
        statements.append(statement_text)

    for i in statement_meta:
        link3 = i.find_all('a')
        source_text = link3[0].text.strip()
        sources.append(source_text)

    for i in target:
        link4 = i.find('div', attrs = {'class':'c-image'}).find('img').get('alt')
        targets.append(link4)


In [40]:
#Loop through 1 to n-1 pages to scrape the data
n=2
for i in range(1,n):
    scrape_website(i) 

In [41]:
#Create DataFrame
data = pd.DataFrame(columns = ['author', 'statement', 'source', 'date', 'target'])
data['author'] = authors
data['statement'] = statements
data['source'] = sources
data['date'] = dates
data['target'] = targets
data

,author,statement,source,date,target
0,Brandon Mulder,Says that shutdowns are “ineffective” and that...,Greg Abbott,"November 27, 2020",false
1,Eric Litke,"""Kyle Rittenhouse used his COVID-19 stimulus c...",Instagram posts,"November 25, 2020",half-true
2,Haley BeMiller,“We passed one of the first (COVID-19) bills i...,Robin Vos,"November 25, 2020",false
3,Samantha Putterman,Video shows movers carrying boxes out of the W...,Instagram posts,"November 24, 2020",false
4,Hayat Norimine,Dominion and Smartmatic “have closed up shop” ...,Facebook posts,"November 24, 2020",false
5,Ciara O'Rourke,“Pennsylvania just banned alcohol sales.”,Viral image,"November 24, 2020",barely-true
6,Ciara O'Rourke,Photo shows “Biden’s inaugural stage being bui...,Viral image,"November 24, 2020",false
7,Ciara O'Rourke,"Says Donald Trump’s sister tweeted, “My brothe...",Viral image,"November 24, 2020",false
8,Hayat Norimine,Democrats “spent 4 years refusing to acknowled...,Jody Hice,"November 24, 2020",false
9,Ciara O'Rourke,North Carolina Gov. Roy Cooper’s “daughter is ...,Facebook posts,"November 23, 2020",false


In [42]:
#Get Binary number or label from the target
def getBinaryNumber(text):
    if text == 'true':
        return 1
    else:
        return 0
def getBinaryLabel(text):
    if text == 'true':
        return 'Real'
    else:
        return 'Fake'


In [43]:
data['News Label Binary Notation'] = data['target'].apply(getBinaryNumber)

In [44]:
data

,author,statement,source,date,target,News Label Binary Notation
0,Brandon Mulder,Says that shutdowns are “ineffective” and that...,Greg Abbott,"November 27, 2020",false,0
1,Eric Litke,"""Kyle Rittenhouse used his COVID-19 stimulus c...",Instagram posts,"November 25, 2020",half-true,0
2,Haley BeMiller,“We passed one of the first (COVID-19) bills i...,Robin Vos,"November 25, 2020",false,0
3,Samantha Putterman,Video shows movers carrying boxes out of the W...,Instagram posts,"November 24, 2020",false,0
4,Hayat Norimine,Dominion and Smartmatic “have closed up shop” ...,Facebook posts,"November 24, 2020",false,0
5,Ciara O'Rourke,“Pennsylvania just banned alcohol sales.”,Viral image,"November 24, 2020",barely-true,0
6,Ciara O'Rourke,Photo shows “Biden’s inaugural stage being bui...,Viral image,"November 24, 2020",false,0
7,Ciara O'Rourke,"Says Donald Trump’s sister tweeted, “My brothe...",Viral image,"November 24, 2020",false,0
8,Hayat Norimine,Democrats “spent 4 years refusing to acknowled...,Jody Hice,"November 24, 2020",false,0
9,Ciara O'Rourke,North Carolina Gov. Roy Cooper’s “daughter is ...,Facebook posts,"November 23, 2020",false,0


In [45]:
#Copy the data to a CSV file
data.to_csv('web_scrape_practice.csv')